<a href="https://colab.research.google.com/github/Ezhil2318/Pandas_Practice/blob/main/pandasPractice_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime,timedelta

In [ ]:
customers_df = pd.DataFrame({
    'customer_id': [101, 102, 103, 104, 105, 106],
    'name': ['Amit', 'Bella', 'Chirag', 'Divya', 'Eshan', 'Farah'],
    'region': ['North', 'South', 'East', 'West', None, 'North'],
    'joined_date': pd.to_datetime(['2023-01-15', '2023-03-10', '2023-05-20', '2023-07-01', '2023-02-28', '2023-08-05']),
})

orders_df = pd.DataFrame({
    'order_id': [5001, 5002, 5003, 5004, 5005, 5006, 5007, 5008, 5009],
    'customer_id': [101, 105, 103, 102, 101, 104, 106, 106, 101],
    'order_amount': [2500, 4000, 1200, 3800, 2750, 6600, 1800, 1900, 3100],
    'order_date': pd.to_datetime(['2023-04-10', '2023-06-15', '2023-08-05', '2023-09-12',
                                  '2023-10-21', '2023-11-01', '2023-08-06', '2023-09-10', '2023-03-01']),
    'product_category': ['Electronics', 'Clothing', 'Home', 'Clothing', 'Electronics',
                         'Home', 'Electronics', 'Electronics', 'Clothing']
})

returns_df = pd.DataFrame({
    'order_id': [5002, 5004, 5010, 5006],  # 5010 is an invalid ID for false positive detection
    'return_reason': ['Defective', 'Size Mismatch', 'No Record', 'Late Delivery'],
    'return_date': pd.to_datetime(['2023-06-10', '2023-08-10', '2023-04-01', '2023-10-20']),
})

SET1

In [ ]:
Oreturn  = returns_df.merge(orders_df,on='order_id',how='inner')
Merged = customers_df.merge(Oreturn,on='customer_id',how='inner')
Merged[Merged['return_date']<Merged['order_date']+pd.Timedelta(days=10)]

,customer_id,name,region,joined_date,order_id,return_reason,return_date,order_amount,order_date,product_category
0,102,Bella,South,2023-03-10,5004,Size Mismatch,2023-08-10,3800,2023-09-12,Clothing
1,104,Divya,West,2023-07-01,5006,Late Delivery,2023-10-20,6600,2023-11-01,Home
2,105,Eshan,None,2023-02-28,5002,Defective,2023-06-10,4000,2023-06-15,Clothing


In [ ]:
all_orders = orders_df.merge(returns_df,on='order_id',how='left')
all_orders = all_orders[(all_orders.product_category=='Electronics')&all_orders.return_date.isna()]
combined = customers_df.merge(all_orders,on='customer_id',how='inner')
combined.groupby('region')['order_amount'].sum().reset_index(name='Total_order')

,region,Total_order
0,North,8950


In [ ]:
cusOrder = customers_df.merge(orders_df,on='customer_id',how='inner')
x = cusOrder.groupby(['customer_id','region'])['order_amount'].mean().reset_index(name='avg_order')
x.pivot_table(index='region',columns='customer_id',values='avg_order',fill_value=0).rank(method='dense')

customer_id,101,102,103,104,106
region,,,,,
East,1.0,1.0,2.0,1.0,1.0
North,2.0,1.0,1.0,1.0,2.0
South,1.0,2.0,1.0,1.0,1.0
West,1.0,1.0,1.0,2.0,1.0


SET2

In [ ]:
orders = orders_df.merge(returns_df,on='order_id',how='left')
orders['False_flags'] = orders.apply(lambda x:(True if x.order_date>x.return_date else False),axis=1)
orders[orders.False_flags==True]

,order_id,customer_id,order_amount,order_date,product_category,return_reason,return_date,False_flags
1,5002,105,4000,2023-06-15,Clothing,Defective,2023-06-10,True
3,5004,102,3800,2023-09-12,Clothing,Size Mismatch,2023-08-10,True
5,5006,104,6600,2023-11-01,Home,Late Delivery,2023-10-20,True


In [ ]:
category = orders_df['product_category'].nunique()
x = orders_df.groupby('customer_id')['product_category'].nunique().reset_index()
x[x.product_category==category]

,customer_id,product_category


In [ ]:
x = customers_df.merge(orders_df,on='customer_id',how='inner')